# Data Gen

In [ ]:
import numpy as np
import pandas as pd

np.random.seed()
N = 1000

# ------------------ Vehicle Routing ------------------
vehicle_routing = pd.DataFrame({
    'distance_km': np.random.uniform(10, 8000, N),
    'planned_speed_kph': np.random.uniform(30, 120, N),
    'actual_speed_kph': np.random.uniform(10, 140, N),
    'eta_hours': np.random.uniform(0.5, 300, N),
    'route_risk_score': np.random.uniform(0, 1, N),
    'fuel_capacity_liters': np.random.uniform(100, 200000, N),
    'fuel_used_liters': np.random.uniform(10, 190000, N),
})
vehicle_routing['speed_variance_kph'] = vehicle_routing['planned_speed_kph'] - vehicle_routing['actual_speed_kph']

# ------------------ Weather ------------------
weather = pd.DataFrame({
    'wave_height_m': np.random.uniform(0, 12, N),
    'wind_speed_kn': np.random.uniform(0, 60, N),
    'wind_direction_deg': np.random.uniform(0, 360, N),
    'visibility_km': np.random.uniform(1, 20, N),
    'storm_probability': np.random.uniform(0, 1, N),
    'precipitation_mm': np.random.uniform(0, 100, N),
})

# ------------------ Load & Capacity ------------------
vessel_load = pd.DataFrame({
    'max_capacity_tons': np.random.uniform(1000, 30000, N),
    'current_load_tons': np.random.uniform(200, 29000, N),
    'num_containers': np.random.randint(50, 2000, N),
    'reefer_containers': np.random.randint(0, 300, N),
    'bulk_cargo_tons': np.random.uniform(0, 5000, N),
    'ballast_water_tons': np.random.uniform(0, 10000, N),
})
vessel_load['load_ratio'] = vessel_load['current_load_tons'] / vessel_load['max_capacity_tons']

# ------------------ Storage & Loading ------------------
cargo_types = ['containers', 'bulk', 'liquid', 'hazardous', 'reefer','vehicles','general_goods']
storage_loading = pd.DataFrame({
    'cargo_type': np.random.choice(cargo_types, N),
    'cargo_weight_tons': np.random.uniform(0.1, 2000, N),
    'loading_speed_tph': np.random.uniform(5, 500, N),
    'unloading_speed_tph': np.random.uniform(5, 500, N),
    'storage_temp_req_C': np.random.uniform(-40, 40, N),
    'containerized': np.random.choice([0, 1], N),
    'storage_capacity_tons': np.random.uniform(10, 50000, N),
    'container_capacity_count': np.random.randint(0, 5000, N),
})
storage_loading['cargo_type_id'] = pd.factorize(storage_loading['cargo_type'])[0]
numeric_storage = storage_loading.drop(columns=['cargo_type'])

# ------------------ Temporal/Port ------------------
temporal_port = pd.DataFrame({
    'arrival_hour': np.random.randint(0, 24, N),
    'arrival_day': np.random.randint(0, 7, N),
    'port_congestion_level': np.random.uniform(0, 1, N),
    'docking_delay_hours': np.random.uniform(0, 48, N),
    'tugboat_availability': np.random.choice([0, 1], N),
})

# ------------------ Combine & Normalize ------------------
datasets = [vehicle_routing, weather, vessel_load, numeric_storage, temporal_port]
DATA_MATRIX = np.hstack([df.values.astype(float) for df in datasets])
DATA_MATRIX = (DATA_MATRIX - DATA_MATRIX.min(axis=0)) / (np.ptp(DATA_MATRIX, axis=0) + 1e-8)

# ------------------ Multi-node Targets ------------------
D_graph = 5
candidate_dims = [7,6,6,7,6]

def generate_targets_per_node(DATA_MATRIX, candidate_dims, D_graph):
    targets = []
    for node_idx in range(D_graph):
        row = DATA_MATRIX[node_idx % DATA_MATRIX.shape[0]]
        dim = candidate_dims[node_idx]
        if len(row) >= dim:
            sampled = row[:dim]
        else:
            sampled = np.pad(row, (0, dim - len(row)), constant_values=0.5)
        targets.append({'target': sampled})
    return targets

synthetic_targets = generate_targets_per_node(DATA_MATRIX, candidate_dims, D_graph)

FEATURE_KEYS = ['Distance', 'Speed', 'Load', 'Capacity', 'TempRequirement', 'Containerization']
FEATURE_TARGET = [[1]*len(FEATURE_KEYS) for _ in range(D_graph)]
METRIC_KEYS = ['TravelTime', 'FuelEfficiency', 'CargoHandling', 'OperationalDelay', 'EnergyConsumption']
METRIC_TARGET = [[1]*len(METRIC_KEYS) for _ in range(D_graph)]

# ------------------ Metrics formulas depending on features ------------------
def compute_metrics_from_features(features):
    # formulas directly from features
    TravelTime = features.get('Distance', 0) / (features.get('Speed', 1e-6))
    FuelEfficiency = features.get('Capacity', 0) / (features.get('Load', 1e-6) + 1e-6)
    CargoHandling = (features.get('loading_speed_tph', features.get('Speed', 1)) / (features.get('Load', 1e-6) + 1e-6))
    OperationalDelay = 1 + np.abs(features.get('Containerization',0) - 0.5) + np.abs(features.get('TempRequirement',0)/40)
    EnergyConsumption = (features.get('Load',0) / (features.get('Capacity',1e-6))) * 0.7
    return {'TravelTime': TravelTime,
            'FuelEfficiency': FuelEfficiency,
            'CargoHandling': CargoHandling,
            'OperationalDelay': OperationalDelay,
            'EnergyConsumption': EnergyConsumption}

class ACO:
    def __init__(self, num_candidates=10, iterations=100, alpha=1.0, beta=2.0, evaporation=0.1):
        self.num_candidates = num_candidates
        self.iterations = iterations
        self.alpha = alpha
        self.beta = beta
        self.evaporation = evaporation

    def optimize(self, features, y=None, metric_mask=None):
        base = np.mean(list(features.values())) if len(features) > 0 else 0.5
        pheromone = np.ones(self.num_candidates)
        best_value = base
        for _ in range(self.iterations):
            candidates = base + 0.1 * np.random.randn(self.num_candidates)
            active_metric_count = sum(metric_mask) if metric_mask is not None else len(METRIC_KEYS)
            fitness = candidates + active_metric_count * 0.01 + np.random.randn(self.num_candidates)*0.001
            weights = (pheromone ** self.alpha) * ((fitness + 1e-8) ** self.beta)
            probs = weights / np.sum(weights)
            chosen_idx = np.random.choice(self.num_candidates, p=probs)
            chosen_value = candidates[chosen_idx]
            pheromone = (1 - self.evaporation) * pheromone
            pheromone[chosen_idx] += self.evaporation * fitness[chosen_idx]
            if chosen_value > best_value:
                best_value = chosen_value
        return best_value

class MetricsEvaluator:
    def __init__(self, data_matrix):
        self.data_matrix = data_matrix
        self.num_nodes = data_matrix.shape[0]
        self.optimizer = ACO()

    def extract_features(self, node_idx):
        idx = node_idx % self.data_matrix.shape[0]
        node_data = self.data_matrix[idx, :]
        defaults = {k:0.5 for k in FEATURE_KEYS}
        features = {}
        for i, key in enumerate(FEATURE_KEYS):
            features[key] = float(node_data[i]) if i < len(node_data) else defaults[key]
        mask = FEATURE_TARGET[node_idx % len(FEATURE_TARGET)]
        features = {k:v for k,v,m in zip(FEATURE_KEYS, features.values(), mask) if m}
        return features

    def compute_node_metrics(self, node_idx, y=None):
        features = self.extract_features(node_idx)
        metric_mask = METRIC_TARGET[node_idx % len(METRIC_TARGET)]
        opt_value = self.optimizer.optimize(features, y=y, metric_mask=metric_mask)
        metric_values = compute_metrics_from_features(features)
        masked_metrics = {k:(metric_values[k]*m) for k,m in zip(METRIC_KEYS, metric_mask)}
        masked_metrics['score'] = sum(masked_metrics.values())
        return masked_metrics


In [ ]:

candidate_dims = [[i]*10 for i in candidate_dims]
candidate_dims

# ACO for the Multiplex

**Maps any metric to any node target explicitly and solves for the data**

# Fuzzy Multiplex


In [ ]:


import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
from scipy.optimize import minimize
import numpy as np

# ---------------- CONFIG ----------------nsion

inner_archive_size = 80
inner_offspring = 40
outer_archive_size = 40
outer_offspring = 40
inner_iters_per_outer = 50
outer_generations = 2
outer_cost_limit = 10000
inner_learning = 0.1
gamma_interlayer = 1
top_k = 263
seed = np.random.seed()

new_DATA_MATRIX = np.random.rand(D_graph, DATA_MATRIX.shape[1])



class InterLayer:
    def __init__(self, D_graph, max_inner_dim, inter_dim=None, edge_threshold=0.02, gamma=1.0, seed=42,metrics=METRIC_KEYS):
        np.random.seed(seed)
        self.D_graph = D_graph
        self.edge_threshold = edge_threshold
        self.gamma = gamma
        self.inter_dim = inter_dim[0] if isinstance(inter_dim, list) else (inter_dim if inter_dim is not None else max_inner_dim[0] if isinstance(max_inner_dim, list) else max_inner_dim)
        self.max_input = 2 * (max_inner_dim[0] if isinstance(max_inner_dim, list) else max_inner_dim)
                    # Initialize weights proportional to synthetic correlation between nodes
        self.weights = {}
        self.bias = {}
        self.MK = metrics
        for i in range(D_graph):
            for j in range(D_graph):
                if i != j:
                    # small random + slight bias towards correlation
                    w_init = np.random.uniform(-0.1, 0.1, (self.inter_dim, self.max_input))
                    self.weights[(i,j)] = w_init
                    self.bias[(i,j)] = np.zeros(self.inter_dim)

    def compute_edge_activation(self, i, j, nested_reps):
        concat = np.concatenate([nested_reps[i], nested_reps[j]])
        concat = np.pad(concat, (0, max(0, self.max_input - len(concat))))[:self.max_input]

        # Normalize input to improve correlation
        concat = (concat - np.mean(concat)) / (np.std(concat) + 1e-12)

        # Compute activation
        v = self.weights[(i,j)].dot(concat) + self.bias[(i,j)]

        # Scale by correlation strength with input signals
        input_strength = np.clip(np.mean(np.abs(concat)), 0, 1)
        v = v * input_strength

        return 1 / (1 + np.exp(-v))

    def build_activations(self, Gmat, nested_reps):
        acts = {}
        for i in range(self.D_graph):
            for j in range(self.D_graph):
                if i == j:
                    continue
                if abs(Gmat[i,j]) > self.edge_threshold:
                    acts[(i,j)] = self.compute_edge_activation(i, j, nested_reps)
        return acts

    @staticmethod
    def pairwise_squared_corr(acts):
        if len(acts) < 2:
            return 0.0
        A = np.stack(list(acts.values()))
        A_centered = A - A.mean(axis=1, keepdims=True)
        stds = np.sqrt(np.sum(A_centered**2, axis=1) / (A.shape[1]-1) + 1e-12)
        cov = A_centered @ A_centered.T / (A.shape[1]-1)
        corr = cov / (np.outer(stds, stds) + 1e-12)
        np.fill_diagonal(corr, 0)
        return float((corr**2).sum())

    def mi_for_graph(self, Gmat, nested_reps):
        acts = self.build_activations(Gmat, nested_reps)
        if not acts:
            return 0.0
        return self.gamma * self.pairwise_squared_corr(acts)

    def correlate_shrink_interlayer(self, fmt_bounds=None, interaction_tensor=None, metrics_keys=None, verbose=True):
        """
        Compute Pearson correlation per node & metric between:
            - shrink factor (adaptive FMT)
            - mean outgoing inter-layer activations
        Returns: {node_idx: {metric: {'r':..., 'p':...}}}
        """
        from scipy.stats import pearsonr

        if metrics_keys is None:
            metrics_keys = self.MK
        D = self.D_graph

        # 1. Compute FMT bounds if not given
        if fmt_bounds is None:
            fmt_bounds = self.compute_fmt_with_bounds_adaptive(top_k=top_k)

        # 2. Get inter-layer activations if not provided
        if interaction_tensor is None:
            interaction_tensor = self.print_interactions(return_tensor=True, verbose=False)

        inter_mean = interaction_tensor.mean(axis=2)  # (D,D)
        shrink_factors = self.compute_fmt_shrink_factor(fmt_bounds, metrics_keys)  # (D, num_metrics)

        correlations = {}

        for i in range(D):
            correlations[i] = {}
            for k, key in enumerate(metrics_keys):
                # FMT shrink for node i (broadcasted across outgoing edges)
                shrink_vec = shrink_factors[i, k] * np.ones(D)
                # Outgoing inter-layer activations from node i
                inter_vec = inter_mean[i, :]
                # Remove self-loop
                mask = np.arange(D) != i
                shrink_vec = shrink_vec[mask]
                inter_vec = inter_vec[mask]

                # Compute Pearson correlation
                if np.std(inter_vec) > 1e-8:  # valid correlation
                    r, p = pearsonr(shrink_vec, inter_vec)
                else:
                    r, p = 0.0, 1.0  # no variability

                correlations[i][key] = {'r': r, 'p': p}
                if verbose:
                    print(f"Node {i} | {key} shrink vs inter-layer: r={r:.3f}, p={p:.3e}")

        return correlations



# ---------------- UNIFIED ACOR MULTIPLEX ----------------
class Fuzzy_Hierarchical_Multiplex:
    def __init__(self, candidate_dims, D_graph, inner_archive_size, inner_offspring,
                 outer_archive_size, outer_offspring, synthetic_targets, inner_learning,
                 gamma_interlayer=1.0, causal_flag=True,metrics=METRIC_KEYS):
        self.candidate_dims = candidate_dims
        self.D_graph = D_graph
        self.inner_archive_size = inner_archive_size
        self.inner_offspring = inner_offspring
        self.outer_archive_size = outer_archive_size
        self.outer_offspring = outer_offspring
        self.synthetic_targets = synthetic_targets
        self.inner_learning = inner_learning
        self.causal_flag = causal_flag
        self.best_dim_per_node = [len(t)-1 for t in synthetic_targets]  # last element as best dim
        self.MK = metrics
        self.MKI = metrics+['score']
        self.nested_reps = [np.zeros(c[0]) for c in candidate_dims]
      #  self.best_dim_per_node = [candidate_dims[0] for _ in range(D_graph)]
        self.inter_layer = InterLayer(D_graph, max_inner_dim=max(candidate_dims), gamma=gamma_interlayer)
        self.chosen_Gmat = np.random.uniform(-0.5,0.5,(D_graph,D_graph))
        np.fill_diagonal(self.chosen_Gmat,0)
        self.l2_before, self.l2_after = [], []

    # ---------- INNER LOOP (FCM) ----------
    def run_inner(self, node_idx, target, D_fcm,
              steps=100, lr_x=0.001, lr_y=0.001, lr_W=0.001,
              decorrelate_metrics=True):

        # --- Initialize activations ---
        x = target.copy()#np.random.uniform(0, 1, D_fcm)

        y = np.random.uniform(-0.6, 0.6, (D_fcm))

        # Pad target for L2 computation
        target_padded = np.pad(target, (0, len(self.nested_reps[node_idx]) - len(target)),
                            mode='constant', constant_values=0.5)
        self.l2_before.append(np.linalg.norm(self.nested_reps[node_idx] - target_padded))

        # --- FCM updates ---
        W = np.random.uniform(-0.6, 0.6, (D_fcm, D_fcm))
        np.fill_diagonal(W, 0)

        for _ in range(steps):
            z = y.dot(W) + x
            Theta_grad_z = z - target
            Theta_grad_x = Theta_grad_z
            Theta_grad_y = Theta_grad_z.dot(W.T)
            Theta_grad_W = np.outer(y, Theta_grad_z)
         #   Theta_grad_z = 2 * z - target_padded
          #  Theta_grad_x = W.T @ Theta_grad_z + (y + 1)

           # Theta_grad_y = x + 1
            #Theta_grad_W = np.outer(Theta_grad_z, x)

            x -= lr_x * np.clip(Theta_grad_x, -0.05, 0.05)
            y -= lr_y * np.clip(Theta_grad_y, -0.05, 0.05)
            W -= lr_W * np.clip(Theta_grad_W, -0.01, 0.01)

            x = np.clip(x, 0, 1)
            y = np.clip(y, 0, 1)
            np.fill_diagonal(W, 0)
            W = np.clip(W, -1, 1)

        # Pad FCM output to max dim for nested_reps
        x_padded = np.pad(x, (0, len(self.nested_reps[node_idx]) - len(x)),
                        mode='constant', constant_values=0.5)
        self.nested_reps[node_idx] = x_padded
        self.l2_after.append(np.linalg.norm(x_padded - target_padded))

        # --- Compute metrics ONCE ---
        metrics_evaluator = MetricsEvaluator(DATA_MATRIX)
        metrics = metrics_evaluator.compute_node_metrics(node_idx, y=x)

        # --- Compute MI score for inter-layer ---
        mi_score = self.inter_layer.mi_for_graph(self.chosen_Gmat, self.nested_reps)

        return x, y, W, mi_score, metrics

    def run_outer(self):
        """
        Compute node contributions and outer score with normalized weights.
        Uses raw node metrics + FMT sums, plus correlation penalty.
        Weighted FMT and best_weights are stored as attributes.
        """
        node_metrics_list = self.capped_node_metrics
        raw_scores = np.array([m['score'] for m in node_metrics_list])
        D = self.D_graph

        # --- Compute Fuzzy Metric Tensor ---
        fuzzy_tensor = self.compute_fuzzy_metric_tensor(normalize=False)  # no normalization
        gamma = self.inter_layer.gamma

        # --- Compute per-node fuzzy sums ---
        fmt_node_sums = np.array([
            fuzzy_tensor[i,:,:].sum() - fuzzy_tensor[i,i,:].sum()
            for i in range(D)
        ])

        # --- Prepare for optimization ---
        interaction_tensor = self.print_interactions(return_tensor=True, verbose=False)
        fmt_mean = fuzzy_tensor.mean(axis=2)
        inter_mean = interaction_tensor.mean(axis=2)

        lambda_reg = 0.05

        def objective(weights):
            node_contrib = weights * (raw_scores + gamma * fmt_node_sums)

            # correlation penalty
            corr_penalty = 0.0
            for i in range(D):
                fmt_vec = fmt_mean[i,:]
                inter_vec = inter_mean[i,:]
                if np.std(fmt_vec) > 1e-8 and np.std(inter_vec) > 1e-8:
                    corr_penalty += abs(np.corrcoef(fmt_vec, inter_vec)[0,1])**2
            corr_penalty /= D

            # regularization: keep weights near uniform
            reg_penalty = lambda_reg * np.sum((weights - 1.0/D)**2)

            return -(node_contrib.sum() - corr_penalty - reg_penalty)

        # bounds: weights positive
        bounds = [(0.1, 1.0) for _ in range(D)]
        # constraint: sum(weights) = 1
        cons = {'type': 'eq', 'fun': lambda w: np.sum(w) - 1.0}

        from scipy.optimize import minimize
        result = minimize(objective, x0=np.ones(D)/D, bounds=bounds, constraints=cons, method='SLSQP')
        best_weights = result.x

        # --- compute final contributions ---
        node_contributions = best_weights * (raw_scores + gamma * fmt_node_sums)

        # --- weighted FMT ---
        weighted_fmt = fuzzy_tensor.copy()
        for i in range(D):
            weighted_fmt[i,:,:] *= best_weights[i]  # scale row by node weight

        # --- compute correlation penalty separately ---
        corr_penalty = 0.0
        for i in range(D):
            fmt_vec = fmt_mean[i,:]
            inter_vec = inter_mean[i,:]
            if np.std(fmt_vec) > 1e-8 and np.std(inter_vec) > 1e-8:
                corr_penalty += abs(np.corrcoef(fmt_vec, inter_vec)[0,1])**2
        corr_penalty /= D

        combined_score = node_contributions.sum() - corr_penalty

        # Save attributes for later use
        self.node_score_contributions = node_contributions
        self.correlation_penalty = corr_penalty
        self.best_weights = best_weights
        self.weighted_fmt = weighted_fmt

        return node_metrics_list, combined_score, node_contributions





    def run(self, outer_generations=outer_generations):
        final_metrics = None

        for gen in range(outer_generations):
            mi_scores = []
            node_metrics_list = []

            for node_idx in range(self.D_graph):
                # Full target vector for this node
                full_target = self.synthetic_targets[node_idx]['target']

                # Use the candidate dimension assigned to this node
                D_fcm = self.candidate_dims[node_idx][0]  # get the integer
                target = full_target[:D_fcm]


                # Run inner FCM
                _, _, _, mi_score, metrics = self.run_inner(node_idx, target, D_fcm)
                mi_scores.append(mi_score)
                node_metrics_list.append(metrics)


            # --- Outer loop uses decorrelated metrics ---
            self.capped_node_metrics = node_metrics_list
            _, capped_score, node_contributions = self.run_outer()  # uses self.capped_node_metrics

            print(f"\n--- Generation {gen} Metrics ---")
            for i, m in enumerate(node_metrics_list):
                print(f"Node {i} | " + " | ".join([f"{k}: {v:.2f}" for k,v in m.items()]))

            print(f"\n--- Generation {gen} Node Contributions ---")
            for i, c in enumerate(node_contributions):
                print(f"Node {i}: Contribution = {c:.4f}")

            print(f"Outer Score (capped): {capped_score:.3f}")
            # --- UPDATE BEST NESTED REPS ---
            if not hasattr(self, 'best_nested_reps'):
                self.best_nested_reps = [rep.copy() for rep in self.nested_reps]
                self.best_scores = [m['score'] for m in node_metrics_list]
            else:
                for i in range(self.D_graph):
                    old_score = self.best_scores[i]
                    new_score = node_metrics_list[i]['score']
                    if new_score > old_score:
                        self.best_scores[i] = new_score
                        self.best_nested_reps[i] = self.nested_reps[i].copy()

            final_metrics = node_metrics_list

        return final_metrics


    # ---------- VISUALIZATIONS ----------
    # ---------- VISUALIZATIONS ----------
    def plot_pointwise_minmax_elite(self, top_k=21):
        plt.figure(figsize=(14,3))
        for i in range(self.D_graph):
            # Node's actual dimension
            dim_i = self.candidate_dims[i][0]  # ✅ integer
            base = self.nested_reps[i][:dim_i]  # slice to candidate dim
            reps = np.clip(base + np.random.normal(0,0.05,(top_k,len(base))),0,1)
            y_min, y_max = reps.min(axis=0), reps.max(axis=0)
            y_sel = base

            # True target for this node, sliced to candidate dim
            y_true = self.synthetic_targets[i]['target'][:len(y_sel)]
            if len(y_true) < len(y_sel):
                y_true = np.pad(y_true, (0, len(y_sel)-len(y_true)), "constant")
            else:
                y_true = y_true[:len(y_sel)]

            plt.subplot(1,self.D_graph,i+1)
            plt.fill_between(range(len(y_min)),y_min,y_max,color='skyblue',alpha=0.4,label='Elite Interval')
            plt.plot(y_sel,'k-',lw=2,label='Estimated')
            plt.plot(y_true,'r--',lw=2,label='True')
            plt.ylim(0,1.05)
            plt.title(f"Node {i+1}")
            if i==0: plt.legend()
        plt.tight_layout()
        plt.show()


    def plot_nested_activations(self):
        plt.figure(figsize=(12,3))
        for i,rep in enumerate(self.nested_reps):
            dim_i = self.candidate_dims[i][0]
            rep_i = rep[:dim_i]  # slice to candidate dim
            plt.subplot(1,self.D_graph,i+1)
            plt.bar(range(len(rep_i)), rep_i, color=plt.cm.plasma(rep_i))
            plt.ylim(0,1)
            plt.title(f"Node {i+1}")
        plt.tight_layout()
        plt.show()


    def plot_outer_fuzzy_graph(self):
        G = nx.DiGraph()
        for i in range(self.D_graph): G.add_node(i)
        for i in range(self.D_graph):
            for j in range(self.D_graph):
                if i!=j and abs(self.chosen_Gmat[i,j])>0.02:
                    G.add_edge(i,j,weight=self.chosen_Gmat[i,j])
        node_sizes = [self.best_dim_per_node[i]*200 for i in range(self.D_graph)]
        edge_colors = ['green' if d['weight']>0 else 'red' for _,_,d in G.edges(data=True)]
        edge_widths = [abs(d['weight'])*3 for _,_,d in G.edges(data=True)]
        pos = nx.circular_layout(G)
        plt.figure(figsize=(6,6))
        nx.draw(G,pos,node_size=node_sizes,node_color='skyblue',
                edge_color=edge_colors,width=edge_widths,arrows=True,with_labels=True)
        plt.title("Outer Fuzzy Multiplex Graph")
        plt.show()
# ---------------- INTERACTIONS INSPECTOR ----------------

    def print_interactions(self, return_tensor=True, verbose=True):
            D_graph = self.D_graph
            inter_dim = self.inter_layer.inter_dim
            inter_tensor = np.zeros((D_graph, D_graph, inter_dim))

            acts = self.inter_layer.build_activations(self.chosen_Gmat, self.nested_reps)
            if not acts:
                if verbose:
                    print("No active edges above threshold.")
                return inter_tensor if return_tensor else None

            for (i, j), vec in acts.items():
                inter_tensor[i, j, :] = vec
                if verbose:
                    act_str = ", ".join([f"{v:.3f}" for v in vec])
                    print(f"Node {i} -> Node {j}: [{act_str}]")
            return inter_tensor if return_tensor else None

        # Move these outside of print_interactions (class-level)
    def print_l2_summary(self):
            print("\nL2 Distances to Target per Node:")
            for idx, (before, after) in enumerate(zip(self.l2_before, self.l2_after)):
                print(f"Node {idx}: Before={before:.4f}, After={after:.4f}")

    def compute_fuzzy_metric_tensor(self, normalize=True, verbose=False):
            """
            Computes a Fuzzy Metric Tensor (D_graph x D_graph x num_metrics)
            using current nested reps and node metrics.
            Each slice [i,j,:] represents metrics of node j (optionally weighted by Gmat[i,j])
            """
            metrics_keys = self.MK
            D = self.D_graph
            num_metrics = len(metrics_keys)
            tensor = np.zeros((D, D, num_metrics))

            metrics_evaluator = MetricsEvaluator(DATA_MATRIX)

            node_metrics = []
            for i, rep in enumerate(self.nested_reps):
                metrics = metrics_evaluator.compute_node_metrics(i, y=rep)
                node_metrics.append(np.array([metrics[k] for k in metrics_keys]))
            node_metrics = np.array(node_metrics)  # (D, num_metrics)

            for i in range(D):
                for j in range(D):
                    if i==j:
                        tensor[i,j,:] = node_metrics[j]
                    else:
                        weight = np.clip(abs(self.chosen_Gmat[i,j]), 0, 1)
                        tensor[i,j,:] = weight * node_metrics[j]

            if normalize:
                tensor = (tensor - tensor.min()) / (tensor.max() - tensor.min() + 1e-12)

            if verbose:
                print("Fuzzy Metric Tensor shape:", tensor.shape)

            return tensor



    def compute_fmt_shrink_factor(self, fmt_bounds, metrics_keys=None):
        """
        Returns shrink factor per node and metric.
        shrink_factor = 1 - (current_interval / original_interval)
        """
        if metrics_keys is None:
            metrics_keys = self.MK

        D = self.D_graph
        num_metrics = len(metrics_keys)
        shrink_factors = np.zeros((D, num_metrics))

        for i in range(D):
            for k in range(num_metrics):
                lower, upper = fmt_bounds[i, i, k, 0], fmt_bounds[i, i, k, 1]  # self-node interval
                interval_width = upper - lower + 1e-12  # normalized [0,1]
                shrink_factors[i, k] = 1 - interval_width  # more shrink = higher value

        return shrink_factors

    def compute_fmt_with_bounds_adaptive(self, top_k=21, max_shrink=0.5, metrics_keys=None):
        """
        Computes FMT bounds using pointwise min/max across elite solutions,
        and applies dynamic adaptive shrinking where variability is low.
        Returns tensor shape (D,D,num_metrics,2) [lower, upper].
        """
        if metrics_keys is None:
            metrics_keys = self.MK

        D = self.D_graph
        num_metrics = len(metrics_keys)
        tensor_bounds = np.zeros((D, D, num_metrics, 2))
        metrics_evaluator = MetricsEvaluator(DATA_MATRIX)

        variability = np.zeros((D, num_metrics))

        # Step 1: compute bounds from perturbations
        for i in range(D):
            base = self.nested_reps[i]
            reps = np.clip(base + np.random.normal(0, 0.05, (top_k, len(base))), 0, 1)
            metrics_matrix = np.zeros((top_k, num_metrics))
            for idx, rep in enumerate(reps):
                m = metrics_evaluator.compute_node_metrics(i, y=rep)
                metrics_matrix[idx, :] = [m[k] for k in metrics_keys]

            lower_i = metrics_matrix.min(axis=0)
            upper_i = metrics_matrix.max(axis=0)
            tensor_bounds[i, :, :, 0] = lower_i[np.newaxis, :]  # broadcast to all j
            tensor_bounds[i, :, :, 1] = upper_i[np.newaxis, :]
            variability[i, :] = metrics_matrix.std(axis=0)

        # Step 2: adaptive shrinking
        for i in range(D):
            for j in range(D):
                for k in range(num_metrics):
                    lower, upper = tensor_bounds[i,j,k,0], tensor_bounds[i,j,k,1]
                    mean = (lower + upper)/2
                    var_norm = min(1.0, variability[i,k]/(upper-lower + 1e-12))
                    shrink_factor = max_shrink * (1 - var_norm)
                    tensor_bounds[i,j,k,0] = mean - shrink_factor*(mean - lower)
                    tensor_bounds[i,j,k,1] = mean + shrink_factor*(upper - mean)

        return tensor_bounds



    def plot_fuzzy_metric_tensor_heatmaps(self, fuzzy_tensor=None, metrics_keys=METRIC_KEYS):
        """
        Plot a heatmap panel for each metric in the FMT.
        Rows: source node i
        Columns: target node j
        """
        if fuzzy_tensor is None:
            fuzzy_tensor = self.compute_fuzzy_metric_tensor(normalize=True)

        D = self.D_graph
        num_metrics = len(metrics_keys)

        fig, axes = plt.subplots(1, num_metrics, figsize=(4*num_metrics,4))
        if num_metrics == 1: axes = [axes]

        for k, key in enumerate(metrics_keys):
            data = fuzzy_tensor[:,:,k]
            im = axes[k].imshow(data, cmap='viridis', vmin=0, vmax=1)
            for i in range(D):
                for j in range(D):
                    axes[k].text(j,i,f"{data[i,j]:.2f}",ha='center',va='center',color='white',fontsize=9)
            axes[k].set_xticks(range(D))
            axes[k].set_yticks(range(D))
            axes[k].set_xticklabels([f'Node {j}' for j in range(D)])
            axes[k].set_yticklabels([f'Node {i}' for i in range(D)])
            axes[k].set_title(f'FMT - {key}')

        fig.colorbar(im, ax=axes, orientation='vertical', fraction=0.025, pad=0.04, label='Normalized Metric Value')
        plt.tight_layout()
        plt.show()

    def compute_fmt_with_elite_bounds(self, top_k=21):
        """
        Computes FMT bounds using pointwise min/max across elite solutions.
        Returns tensor shape (D,D,num_metrics,2) [lower, upper].
        """
        metrics_keys = self.MK
        D = self.D_graph
        num_metrics = len(metrics_keys)
        tensor_bounds = np.zeros((D,D,num_metrics,2))

        metrics_evaluator = MetricsEvaluator(DATA_MATRIX)

        for i in range(D):
            # Generate top_k perturbations around current nested_rep (like in plot_pointwise_minmax_elite)
            base = self.nested_reps[i]
            reps = np.clip(base + np.random.normal(0,0.05,(top_k,len(base))),0,1)

            # Compute node metrics for each perturbed solution
            metrics_matrix = np.zeros((top_k, num_metrics))
            for idx, rep in enumerate(reps):
                m = metrics_evaluator.compute_node_metrics(i, y=rep)
                metrics_matrix[idx,:] = [m[k] for k in metrics_keys]

            # Compute pointwise min/max across elite solutions
            lower_i = metrics_matrix.min(axis=0)
            upper_i = metrics_matrix.max(axis=0)

            # Fill bounds tensor for all source nodes (i->j)
            for j in range(D):
                tensor_bounds[i,j,:,0] = lower_i
                tensor_bounds[i,j,:,1] = upper_i

        return tensor_bounds


    def plot_fmt_with_bounds(self, fmt_tensor_bounds):
        D = self.D_graph
        metrics_keys = self.MK
        M = len(metrics_keys)

        # Extract lower/upper for each metric and compute mean
        mean_vals = (fmt_tensor_bounds[:, :, :, 0] + fmt_tensor_bounds[:, :, :, 1]) / 2  # (D,D,M)

        # Apply outer-loop weights if available
        if hasattr(self, 'best_weights'):
            for i in range(D):
                mean_vals[i, :, :] *= self.best_weights[i]

        # Take any column (say j=0) for unique row values
        unique_mean = mean_vals[:, 0, :]  # shape (D,M)

        # Plotting
        fig, ax = plt.subplots(figsize=(1.2*M + 4, 0.35*D + 4))
        im = ax.imshow(unique_mean, cmap='viridis', aspect='auto', vmin=0, vmax=unique_mean.max())

        # Annotate heatmap with weighted mean values
        for i in range(D):
            for k in range(M):
                ax.text(k, i, f"{unique_mean[i,k]:.2f}", ha='center', va='center', color='white', fontsize=8)

        # Label axes
        ax.set_xticks(range(M))
        ax.set_xticklabels(metrics_keys, rotation=45, ha='right')
        ax.set_yticks(range(D))
        ax.set_yticklabels([f"Node {i}" for i in range(D)])

        ax.set_title("Weighted FMT (Outer Loop Weights)")
        fig.colorbar(im, ax=ax, label='Weighted Mean Metric Value')
        plt.tight_layout()
        plt.show()

    def plot_node_score_contribution(self, metrics_keys=METRIC_KEYS):
        """
        Plot per-node total score contribution:
            - 3 panels: Raw, FMT (interaction), Total
            - Diagonal represents raw contributions
            - FMT scaled by best_weights if available
            - Annotated cells
            - Normalized across metrics for consistent visualization
        """
        D = self.D_graph
        node_contributions = np.array(self.node_score_contributions)

        # --- FMT contribution ---
        if hasattr(self, 'weighted_fmt'):
            fuzzy_tensor = np.array(self.weighted_fmt)
        else:
            fuzzy_tensor = self.compute_fuzzy_metric_tensor(normalize=True)

        # Normalize across entire tensor for plotting
        fuzzy_tensor_norm = (fuzzy_tensor - fuzzy_tensor.min()) / (fuzzy_tensor.max() - fuzzy_tensor.min() + 1e-12)
        fmt_matrix = fuzzy_tensor_norm.sum(axis=2)  # sum over metrics
        np.fill_diagonal(fmt_matrix, 0)

        # --- Raw matrix on diagonal ---
        raw_matrix = np.zeros((D,D))
        np.fill_diagonal(raw_matrix, node_contributions)

        # --- Total contribution ---
        total_matrix = raw_matrix + fmt_matrix

        # --- Global min/max for color scale ---
        global_min, global_max = total_matrix.min(), total_matrix.max()

        # --- Plot ---
        fig, axes = plt.subplots(1, 3, figsize=(15, 4))
        matrices = [raw_matrix, fmt_matrix, total_matrix]
        titles = ["Raw Node Contribution", "Normalized FMT Contribution", "Total Contribution"]

        for ax, mat, title in zip(axes, matrices, titles):
            im = ax.imshow(mat, cmap='viridis', vmin=0, vmax=1)  # normalized
            for i in range(D):
                for j in range(D):
                    ax.text(j, i, f"{mat[i,j]:.2f}", ha='center', va='center', color='white', fontsize=8)
            ax.set_title(title)
            ax.set_xticks(range(D))
            ax.set_xticklabels([f"Node {i+1}" for i in range(D)])
            ax.set_yticks(range(D))
            ax.set_yticklabels([f"Node {i+1}" for i in range(D)])

        fig.colorbar(im, ax=axes, orientation='vertical', fraction=0.025, pad=0.04, label='Contribution Value')
        plt.tight_layout()
        plt.show()


    def correlate_fmt_interactions_per_node(self, fmt_bounds=None, interaction_tensor=None, verbose=True):
        """
        Correlate the FMT bounds with inter-layer interactions per node and per metric.
        Returns a dict of shape: {node_idx: {metric: {'r':..., 'p':...}}}.
        """
        from scipy.stats import pearsonr
        import matplotlib.pyplot as plt

        metrics_keys = self.MK
        D = self.D_graph

        # Compute tensors if not provided
        if fmt_bounds is None:
            fmt_bounds = self.compute_fmt_with_elite_bounds(top_k=21)
        if interaction_tensor is None:
            interaction_tensor = self.print_interactions(return_tensor=True, verbose=False)

        # Reduce interaction tensor along inter_dim
        inter_mean = interaction_tensor.mean(axis=2)  # (D,D)

        node_correlations = {}

        for i in range(D):
            node_correlations[i] = {}
            for k, key in enumerate(metrics_keys):
                # FMT bounds for target node j from source i (mean of lower/upper)
                fmt_mean = fmt_bounds[i,:,k,:].mean(axis=1)  # shape (D,)
                # Interaction tensor for edges from node i to j
                inter_vec = inter_mean[i,:]  # shape (D,)
                # Pearson correlation
                corr, pval = pearsonr(fmt_mean, inter_vec)
                node_correlations[i][key] = {'r': corr, 'p': pval}

                if verbose:
                    print(f"Node {i} | {key}: r = {corr:.3f}, p = {pval:.3e}")
                    plt.figure(figsize=(4,3))
                    plt.scatter(fmt_mean, inter_vec, alpha=0.7, edgecolor='k', color='skyblue')
                    plt.xlabel(f"FMT {key} (Node {i} -> others)")
                    plt.ylabel(f"Interaction mean (Node {i} -> others)")
                    plt.title(f"Node {i} | {key} correlation: r={corr:.3f}")
                    plt.grid(True)
                    plt.show()

        return node_correlations

    def correlation_penalty(self, fmt_bounds=None, interaction_tensor=None):
        """
        Computes a penalty term that is high if per-node FMT metrics correlate with interactions.
        Returns total penalty to subtract from the outer score.
        """
        from scipy.stats import pearsonr

        D = self.D_graph
        metrics_keys =self.MK

        if fmt_bounds is None:
            fmt_bounds = self.compute_fmt_with_elite_bounds(top_k=top_k)
        if interaction_tensor is None:
            interaction_tensor = self.print_interactions(return_tensor=True, verbose=False)

        inter_mean = interaction_tensor.mean(axis=2)
        total_penalty = 0.0

        for i in range(D):
            for k in range(len(metrics_keys)):
                fmt_mean = fmt_bounds[i,:,k,:].mean(axis=1)
                inter_vec = inter_mean[i,:]
                if np.std(fmt_mean) > 1e-8 and np.std(inter_vec) > 1e-8:
                    corr, _ = pearsonr(fmt_mean, inter_vec)
                    total_penalty += abs(corr)  # penalize high correlation

        # normalize by number of nodes × metrics
        total_penalty /= (D * len(metrics_keys))**2
        return total_penalty
    def plot_inner_vs_fmt(self, top_k=21, metrics_keys=METRIC_KEYS):
      """
      Plots, for each node:
      - Left: inner structure (nested activations)
      - Right: FMT row (interactions to other nodes)
      """
      D = self.D_graph
      # Compute FMT tensor if needed
      fmt_tensor = self.compute_fuzzy_metric_tensor(normalize=True)

      for i in range(D):
          # 1. Inner activations (nested reps)
          dim_i = self.candidate_dims[i][0]
          rep_i = self.nested_reps[i][:dim_i]

          # 2. FMT row (interactions from node i to all others)
          fmt_row = fmt_tensor[i,:,:].mean(axis=1)  # average over metrics

          fig, axes = plt.subplots(1, 2, figsize=(10,4))

          # --- Inner activations ---
          axes[0].bar(range(len(rep_i)), rep_i, color=plt.cm.plasma(rep_i))
          axes[0].set_ylim(0,1)
          axes[0].set_title(f'Node {i} Inner Activations')
          axes[0].set_xlabel('Inner Dimension')
          axes[0].set_ylabel('Activation Value')

          # --- FMT row ---
          axes[1].imshow(fmt_row[np.newaxis, :], cmap='viridis', aspect='auto', vmin=0, vmax=1)
          for j, val in enumerate(fmt_row):
              axes[1].text(j, 0, f"{val:.2f}", ha='center', va='center', color='white', fontsize=9)
          axes[1].set_title(f'Node {i} FMT Interactions')
          axes[1].set_yticks([])
          axes[1].set_xticks(range(D))
          axes[1].set_xticklabels([f'Node {j}' for j in range(D)], rotation=45)

          plt.tight_layout()
          plt.show()
    def plot_inner_vs_fmt_as_fcm(self):
      """
      Plots, for each node:
      - Left: inner structure as an FCM graph
      - Right: FMT row as an FCM graph for that node
      """
      import networkx as nx
      import matplotlib.pyplot as plt
      import numpy as np

      D = self.D_graph
      fmt_tensor = self.compute_fuzzy_metric_tensor(normalize=True)

      for src in range(D):
          # --- Inner structure as FCM ---
          dim_i = self.candidate_dims[src][0]
          rep_i = self.nested_reps[src][:dim_i]
          G_inner = nx.DiGraph()
          for n in range(dim_i):
              G_inner.add_node(n, activation=rep_i[n])
          # Inner FCM edges: simple pairwise combination of activations
          for i in range(dim_i):
              for j in range(dim_i):
                  if i != j:
                      weight = (rep_i[i] - rep_i[j])  # example: difference as interaction
                      G_inner.add_edge(i, j, weight=weight)
          pos_inner = nx.circular_layout(G_inner)

          # --- FMT row as FCM ---
          G_fcm = nx.DiGraph()
          node_activations = np.mean(fmt_tensor, axis=2).mean(axis=1)
          for n in range(D):
              G_fcm.add_node(n, activation=node_activations[n])
          for tgt in range(D):
              if src != tgt:
                  seq = fmt_tensor[src, tgt, :]
                  if np.any(seq != 0):
                      G_fcm.add_edge(src, tgt, weight=np.mean(seq))
          pos_fcm = nx.circular_layout(G_fcm)

          # --- Plot side by side ---
          fig, axes = plt.subplots(1, 2, figsize=(12,6))

          # Inner FCM
          node_colors_inner = [G_inner.nodes[n]['activation'] for n in G_inner.nodes()]
          edge_colors_inner = ['green' if G_inner.edges[e]['weight']>0 else 'red' for e in G_inner.edges()]
          edge_widths_inner = [2 + 4*abs(G_inner.edges[e]['weight']) for e in G_inner.edges()]
          nx.draw(G_inner, pos_inner, ax=axes[0], with_labels=True,
                  node_color=node_colors_inner, cmap='plasma',
                  edge_color=edge_colors_inner, width=edge_widths_inner,
                  node_size=400, arrowsize=20)
          axes[0].set_title(f'Node {src} Inner FCM')

          # Node FCM
          node_colors_fcm = [G_fcm.nodes[n]['activation'] for n in G_fcm.nodes()]
          edge_colors_fcm = ['green' if G_fcm.edges[e]['weight']>0 else 'red' for e in G_fcm.edges()]
          edge_widths_fcm = [2 + 4*abs(G_fcm.edges[e]['weight']) for e in G_fcm.edges()]
          nx.draw(G_fcm, pos_fcm, ax=axes[1], with_labels=True,
                  node_color=node_colors_fcm, cmap='plasma',
                  edge_color=edge_colors_fcm, width=edge_widths_fcm,
                  node_size=400, arrowsize=20)
          axes[1].set_title(f'Node {src} FCM Row')

          plt.tight_layout()
          plt.show()


# ---------------- USAGE ----------------
if __name__ == "__main__":
    optimizer = Fuzzy_Hierarchical_Multiplex(
        candidate_dims, D_graph,
        inner_archive_size, inner_offspring,
        outer_archive_size, outer_offspring,
        synthetic_targets,
        inner_learning, gamma_interlayer=0,
        causal_flag=False
    )
    metrics_list = optimizer.run()
    optimizer.plot_pointwise_minmax_elite()
    optimizer.plot_inner_vs_fmt()

    optimizer.plot_nested_activations()
    #optimizer.plot_inner_vs_fmt_as_fcm()
    # Compute FMT with elite bounds
    fmt_elite_bounds = optimizer.compute_fmt_with_elite_bounds(top_k=top_k+10)

# Plot as heatmaps
    optimizer.plot_fmt_with_bounds(fmt_elite_bounds)

    # Compute fuzzy multiplex tensor
    fmt_tensor = optimizer.compute_fuzzy_metric_tensor(normalize=True)

    #optimizer.plot_fuzzy_metric_tensor_heatmaps(fmt_tensor)

    # Compute FMT with bounds (minimax elite intervals)
    #optimizer.plot_node_score_contribution()
    #optimizer.plot_outer_fuzzy_graph()
  #  optimizer.print_interactions()
    tensor = optimizer.print_interactions()

    print("Tensor shape:", tensor.shape,'\n',tensor)
    # Compute tensors first
    fmt_elite_bounds = optimizer.compute_fmt_with_elite_bounds(top_k=top_k)
    interaction_tensor = optimizer.print_interactions(return_tensor=True, verbose=False)


    interaction_tensor = optimizer.print_interactions(return_tensor=True, verbose=False)

    # Get per-node, per-metric correlations
    #node_metric_corrs = optimizer.correlate_fmt_interactions_per_node(
     #   fmt_bounds=fmt_elite_bounds,
      #  interaction_tensor=interaction_tensor
   # )

In [ ]:


import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
D_graph = len(optimizer.nested_reps)
tensor = optimizer.print_interactions(return_tensor=True, verbose=False)

# ---------------- Outer nodes (hubs) ----------------
G_outer = nx.DiGraph()
for i in range(D_graph):
    G_outer.add_node(i)
for i in range(D_graph):
    for j in range(D_graph):
        if i != j and np.any(tensor[i,j,:] != 0):
            # Shift to signed weights: 0.5 -> 0, <0.5 negative, >0.5 positive
            mean_weight = 2 * (np.mean(tensor[i,j,:]) - 0.5)
            G_outer.add_edge(i, j, weight=mean_weight)

# Outer spring layout
pos_outer_2d = nx.circular_layout(G_outer, scale=5)
pos_outer = np.array([[x, y, 0] for x, y in pos_outer_2d.values()])

fig = plt.figure(figsize=(10,8))
ax = fig.add_subplot(111, projection='3d')

# Plot outer nodes
for i in range(D_graph):
    ax.scatter(*pos_outer[i], s=300, color='skyblue')
    ax.text(*pos_outer[i], f'Node {i}', color='black')

# Plot outer edges with positive/negative colors

for i, j, data in G_outer.edges(data=True):
    x_vals = [pos_outer[i,0], pos_outer[j,0]]
    y_vals = [pos_outer[i,1], pos_outer[j,1]]
    z_vals = [pos_outer[i,2], pos_outer[j,2]]

    # Positive = bright green, Negative = bright red
    color = 'green' if data['weight'] > 0 else 'red'
    linewidth = 2 + 4*abs(data['weight'])  # scale width by magnitude
    ax.plot(x_vals, y_vals, z_vals, color=color, linewidth=linewidth)
# ---------------- Inner FCMs (small circular around hub) ----------------
for i, rep in enumerate(optimizer.nested_reps):
    dims = len(rep)
    angle = np.linspace(0, 2*np.pi, dims, endpoint=False)
    radius = 0.8  # small circle
    xs = pos_outer[i,0] + radius * np.cos(angle)
    ys = pos_outer[i,1] + radius * np.sin(angle)
    zs = pos_outer[i,2] + rep  # activation as height

    # Plot inner nodes
    ax.scatter(xs, ys, zs, c=rep, cmap='plasma', s=50)

    # Connect inner nodes in circle
    for k in range(dims):
        ax.plot([xs[k], xs[(k+1)%dims]], [ys[k], ys[(k+1)%dims]], [zs[k], zs[(k+1)%dims]], color='gray', alpha=0.5)

ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Activation')
ax.set_title('Outer Nodes with Inner FCMs (Signed correlations)')
plt.show()


In [ ]:
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt

D_graph = len(optimizer.nested_reps)
tensor = optimizer.print_interactions(return_tensor=True, verbose=False)

# ---------------- Outer nodes (hubs) ----------------

G_outer = nx.DiGraph()
for i in range(D_graph):
  G_outer.add_node(i)
  for i in range(D_graph):
    for j in range(D_graph):
      if i != j and np.any(tensor[i,j,:] != 0):
        mean_weight = 2 * (np.mean(tensor[i,j,:]) - 0.5)
        G_outer.add_edge(i, j, weight=mean_weight)

# Outer circular layout in 2D

    pos_outer = nx.circular_layout(G_outer, scale=5)

fig, ax = plt.subplots(figsize=(10,8))

# Plot outer nodes

for i in range(D_graph):
  x, y = pos_outer[i]
  ax.scatter(x, y, s=300, color='skyblue', zorder=2)
  ax.text(x, y, f'Node {i}', color='black', ha='center', va='center')

# Plot outer edges with positive/negative colors

for i, j, data in G_outer.edges(data=True):
  x_vals = [pos_outer[i][0], pos_outer[j][0]]
  y_vals = [pos_outer[i][1], pos_outer[j][1]]
  color = 'green' if data['weight'] > 0 else 'red'
  linewidth = 2 + 4*abs(data['weight'])
  ax.plot(x_vals, y_vals, color=color, linewidth=linewidth, zorder=1)

# ---------------- Inner FCMs (small circular around hub) ----------------

for i, rep in enumerate(optimizer.nested_reps):
  dims = len(rep)
  angle = np.linspace(0, 2*np.pi, dims, endpoint=False)
  radius = 0.8
  x_center, y_center = pos_outer[i]
  xs = x_center + radius * np.cos(angle)
  ys = y_center + radius * np.sin(angle)

  # Map activation to color
  colors = rep
  ax.scatter(xs, ys, c=colors, cmap='plasma', s=50, zorder=3)

# Connect inner nodes in circle
  for k in range(dims):
        ax.plot([xs[k], xs[(k+1)%dims]], [ys[k], ys[(k+1)%dims]], color='gray', alpha=0.5, zorder=2)

ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_title('Outer Nodes with Inner FCMs (2D View)')
ax.set_aspect('equal')
plt.show()


In [ ]:

import numpy as np

class GDFCMPredictorForward:
    """
    Forward predictor for GDFCM.
    Can handle new input data (DATA_MATRIX) and compute:
        - Node activations
        - Node metrics
        - Inter-layer MI scores
        - Total contributions and outer score
    """
    def __init__(self, trained_gdfcm: Fuzzy_Hierarchical_Multiplex):
        self.gdfcm = trained_gdfcm
        self.D_graph = trained_gdfcm.D_graph
        self.nested_reps = trained_gdfcm.nested_reps
        self.chosen_Gmat = trained_gdfcm.chosen_Gmat
        self.inter_layer = trained_gdfcm.inter_layer

    def predict_node(self, node_idx, data_row):
        """
        Predict metrics and activations for a single node given a new data row.
        """
        # Use stored nested_rep as starting point
        x = self.nested_reps[node_idx].copy()
        # Optional: you could combine with data_row to modify x
        # For now, we just compute metrics using data_row as the input
        metrics_evaluator = MetricsEvaluator(np.array([data_row]))
        metrics = metrics_evaluator.compute_node_metrics(0, y=x)

        # Inter-layer MI
        mi_score = self.inter_layer.mi_for_graph(self.chosen_Gmat, self.nested_reps)

        return {'activations': x, 'metrics': metrics, 'mi_score': mi_score}

    def predict_all_nodes(self, new_data_matrix):
        """
        Predict metrics and activations for all nodes using new data.
        new_data_matrix: shape (D_graph, num_features)
        """
        results = []
        for node_idx in range(self.D_graph):
            data_row = new_data_matrix[node_idx % len(new_data_matrix)]
            node_result = self.predict_node(node_idx, data_row)
            results.append(node_result)
        return results

    def predict_scores(self):
        """
        Compute per-node contributions and total outer score.
        """
        _, total_score, node_contributions = self.gdfcm.run_outer()
        return {'node_contributions': node_contributions, 'total_score': total_score}
# Suppose optimizer is your trained GDFCM
predictor = GDFCMPredictorForward(optimizer)

# New data: same number of nodes, each with same num_features

# Predict all nodes
predictions = predictor.predict_all_nodes(DATA_MATRIX)

for idx, node_pred in enumerate(predictions):
    print(f"Node {idx} Metrics:", node_pred['metrics'])
    print(f"Node {idx} Activations:", node_pred['activations'])
    print(f"Node {idx} MI Score:", node_pred['mi_score'])

# Optional: compute total contributions
score_info = predictor.predict_scores()
print("Node Contributions:", score_info['node_contributions'])
print("Total Score:", score_info['total_score'])

class GDFCMPredictorAdaptive:
    """
    Adaptive forward predictor for GDFCM.
    - Accepts new data per node.
    - Updates activations slightly using a mini inner-loop.
    - Computes metrics, inter-layer MI, and outer score contributions.
    """
    def __init__(self, trained_gdfcm: Fuzzy_Hierarchical_Multiplex, lr_x=0.01, lr_steps=10):
        self.gdfcm = trained_gdfcm
        self.D_graph = trained_gdfcm.D_graph
        self.nested_reps = [rep.copy() for rep in trained_gdfcm.nested_reps]
        self.chosen_Gmat = trained_gdfcm.chosen_Gmat
        self.inter_layer = trained_gdfcm.inter_layer
        self.lr_x = lr_x
        self.lr_steps = lr_steps

    def adapt_node(self, node_idx, data_row):
        """
        Mini inner-loop: slightly update activations based on new data row.
        Handles shape mismatch by projecting new data to node activation dim.
        """
        x = self.nested_reps[node_idx].copy()
        dim = len(x)

        # Simple linear projection of new data to activation space
        if len(data_row) != dim:
            # Use mean pooling to reduce or slice if too large
            if len(data_row) > dim:
                target = data_row[:dim]
            else:
                # Pad with 0.5
                target = np.pad(data_row, (0, dim - len(data_row)), 'constant', constant_values=0.5)
        else:
            target = data_row

        target = np.clip(target, 0, 1)

        # Mini inner-loop update
        for _ in range(self.lr_steps):
            grad = x - target
            x -= self.lr_x * grad
            x = np.clip(x, 0, 1)

        self.nested_reps[node_idx] = x

        # Compute metrics
        metrics_evaluator = MetricsEvaluator(np.array([data_row]))
        metrics = metrics_evaluator.compute_node_metrics(0, y=x)

        # Inter-layer MI
        mi_score = self.inter_layer.mi_for_graph(self.chosen_Gmat, self.nested_reps)

        return {'activations': x, 'metrics': metrics, 'mi_score': mi_score}


    def adapt_all_nodes(self, new_data_matrix):
        """
        Update activations for all nodes given new data matrix.
        """
        results = []
        for node_idx in range(self.D_graph):
            data_row = new_data_matrix[node_idx % len(new_data_matrix)]
            node_result = self.adapt_node(node_idx, data_row)
            results.append(node_result)
        return results

    def compute_outer_score(self):
        """
        Compute node contributions and total outer score using current nested_reps.
        """
        _, total_score, node_contributions = self.gdfcm.run_outer()
        return {'node_contributions': node_contributions, 'total_score': total_score}
# Initialize adaptive predictor
adaptive_predictor = GDFCMPredictorAdaptive(optimizer, lr_x=0.02, lr_steps=15)

# New data

# Adapt activations to new data
predictions = adaptive_predictor.adapt_all_nodes(new_DATA_MATRIX)

for idx, node_pred in enumerate(predictions):
    print(f"Node {idx} Metrics:", node_pred['metrics'])
    print(f"Node {idx} Activations:", node_pred['activations'])
    print(f"Node {idx} MI Score:", node_pred['mi_score'])

# Compute outer node contributions after adaptation
score_info = adaptive_predictor.compute_outer_score()
print("Node Contributions:", score_info['node_contributions'])
print("Total Score:", score_info['total_score'])
import numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import mean_squared_error, mean_absolute_error

pred_metrics = []
true_metrics = []
for idx, data_row in enumerate(new_DATA_MATRIX):
    pred = adaptive_predictor.adapt_node(idx, data_row)
    pred_metrics.append([pred['metrics'][k] for k in METRIC_KEYS])
    true_metrics.append([MetricsEvaluator(new_DATA_MATRIX).compute_node_metrics(idx)[k]
                         for k in METRIC_KEYS])

pred_metrics = np.array(pred_metrics)
true_metrics = np.array(true_metrics)


def evaluate_predictions(true_metrics, pred_metrics, metric_names=None):
    """
    Evaluate multi-metric predictions per node and overall.

    Args:
        true_metrics: np.array, shape (num_nodes, num_metrics)
        pred_metrics: np.array, same shape as true_metrics
        metric_names: list of metric names (optional)

    Returns:
        dict with evaluation metrics
    """
    true_metrics = np.array(true_metrics)

    pred_metrics = np.array(pred_metrics)

    if metric_names is None:
        metric_names = [f'Metric {i}' for i in range(true_metrics.shape[1])]

    num_nodes, num_metrics = true_metrics.shape

    results = {'per_metric': {}, 'overall': {}}

    # Per metric
    for i, name in enumerate(metric_names):
        t = true_metrics[:, i]
        p = pred_metrics[:, i]
        mae = mean_absolute_error(t, p)
        rmse = np.sqrt(mean_squared_error(t, p))
        r2 = r2_score(t, p)
        corr = np.corrcoef(t, p)[0,1]
        mape = np.mean(np.abs((t - p) / (t + 1e-12))) * 100
        results['per_metric'][name] = {
            'MAE': mae,
            'RMSE': rmse,
            'R2': r2,
            'Pearson': corr,
            'MAPE (%)': mape
        }

    # Overall
    mae_overall = mean_absolute_error(true_metrics.flatten(), pred_metrics.flatten())
    rmse_overall = np.sqrt(mean_squared_error(true_metrics.flatten(), pred_metrics.flatten()))
    r2_overall = r2_score(true_metrics.flatten(), pred_metrics.flatten())

    # Cosine similarity (averaged over nodes)
    cos_sim = np.mean([cosine_similarity(t.reshape(1,-1), p.reshape(1,-1))[0,0]
                       for t,p in zip(true_metrics, pred_metrics)])

    results['overall'] = {
        'MAE': mae_overall,
        'RMSE': rmse_overall,
        'R2': r2_overall,
        'CosineSim': cos_sim
    }

    return results
# Suppose true_metrics and pred_metrics have shape (D_graph, 4)
metrics_eval = evaluate_predictions(true_metrics, pred_metrics, metric_names=METRIC_KEYS)

# Print per-metric evaluation
for metric, vals in metrics_eval['per_metric'].items():
    print(f"{metric}: {vals}")

# Print overall evaluation
print("\nOverall metrics:", metrics_eval['overall'])


